In [76]:
import requests  # This line should be executed first.
import pandas as pd
import json
from dateutil import parser

In [77]:
API_KEY = "950e89e18324a1decd3d88f4cc43a085-5f59d5913e0f487ebe5eec4cb06b20c1"
ACCOUNT_ID = "101-001-27981277-001" 
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [78]:
session = requests.Session()

In [79]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})   

In [80]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [81]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [82]:
response = session.get(url, params=None, data=None, headers=None)

In [83]:
response.status_code

200

In [84]:
data = response.json()

In [85]:
instruments_list = data['instruments']

In [86]:
len(instruments_list)

68

In [87]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [88]:
key_i = ['name', 'type', 'displayName', 'pipLocation',
        'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [89]:
#loop through instruments list and print name 
instruments_dict = {}
for i in instruments_list:
    key = i['name'] 
    instruments_dict[key] = { k: i[k] for k in key_i } #loop through values in list, k is set to teach value, value for keys is from instrumenst list

In [90]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [91]:
with open("../code/data/instruments.json", "w") as f:
    #Write dictionary into json format
    f.write(json.dumps(instruments_dict, indent=2))

In [154]:
def fetch_candles(pair_name, count=10, granularity="H1" ):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, data=None, headers=None)   
    data = response.json()
    
    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame() #use return pd.DataFrame() not .empty

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']#keys for looping 


    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
    
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"]=float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    if len(data) == 0:
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../code/data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")


In [155]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [156]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 10 candles, 2024-01-18 06:00:00+00:00 2024-01-19 18:00:00+00:00


In [157]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [158]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4001 candles, 2023-05-30 04:00:00+00:00 2024-01-19 21:00:00+00:00
EUR_USD H4 4001 candles, 2021-06-28 01:00:00+00:00 2024-01-19 18:00:00+00:00
EUR_GBP H1 4001 candles, 2023-05-30 04:00:00+00:00 2024-01-19 21:00:00+00:00
EUR_GBP H4 4001 candles, 2021-06-28 01:00:00+00:00 2024-01-19 18:00:00+00:00
EUR_JPY H1 4001 candles, 2023-05-30 04:00:00+00:00 2024-01-19 21:00:00+00:00
EUR_JPY H4 4001 candles, 2021-06-25 17:00:00+00:00 2024-01-19 18:00:00+00:00
EUR_CHF H1 4001 candles, 2023-05-30 04:00:00+00:00 2024-01-19 21:00:00+00:00
EUR_CHF H4 4001 candles, 2021-06-28 01:00:00+00:00 2024-01-19 18:00:00+00:00
EUR_NZD H1 4001 candles, 2023-05-30 05:00:00+00:00 2024-01-19 21:00:00+00:00
EUR_NZD H4 4001 candles, 2021-06-28 09:00:00+00:00 2024-01-19 18:00:00+00:00
EUR_CAD H1 4001 candles, 2023-05-30 04:00:00+00:00 2024-01-19 21:00:00+00:00
EUR_CAD H4 4001 candles, 2021-06-28 01:00:00+00:00 2024-01-19 18:00:00+00:00
EUR_AUD H1 4001 candles, 2023-05-30 04:00:00+00:00 2024-01-19 21:00:00+00:00

In [147]:
#data[0]

In [144]:
#df.info()

In [145]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-01-19 02:00:00+00:00,2643,1.08854,1.08898,1.08826,1.08882,1.08847,1.08890,1.08818,1.08874,1.08862,1.08905,1.08834,1.08889
1,2024-01-19 03:00:00+00:00,1644,1.08884,1.08894,1.08855,1.08890,1.08877,1.08887,1.08847,1.08883,1.08892,1.08902,1.08863,1.08898
2,2024-01-19 04:00:00+00:00,1049,1.08892,1.08895,1.08855,1.08865,1.08883,1.08888,1.08847,1.08857,1.08900,1.08903,1.08862,1.08873
3,2024-01-19 05:00:00+00:00,1935,1.08864,1.08871,1.08796,1.08810,1.08857,1.08864,1.08789,1.08802,1.08872,1.08879,1.08804,1.08817
4,2024-01-19 06:00:00+00:00,2385,1.08806,1.08825,1.08755,1.08810,1.08799,1.08816,1.08748,1.08803,1.08814,1.08834,1.08762,1.08818
5,2024-01-19 07:00:00+00:00,4466,1.08818,1.08864,1.08746,1.08794,1.08811,1.08857,1.08739,1.08787,1.08825,1.08871,1.08753,1.08802
6,2024-01-19 08:00:00+00:00,4291,1.08796,1.08820,1.08680,1.08774,1.08789,1.08813,1.08673,1.08766,1.08804,1.08828,1.08688,1.08781
7,2024-01-19 09:00:00+00:00,4339,1.08772,1.08850,1.08714,1.08831,1.08765,1.08843,1.08707,1.08823,1.08779,1.08858,1.08722,1.08839
8,2024-01-19 10:00:00+00:00,3617,1.08828,1.08853,1.08731,1.08810,1.08821,1.08846,1.08723,1.08803,1.08836,1.08860,1.08739,1.08818
9,2024-01-19 11:00:00+00:00,3119,1.08813,1.08932,1.08794,1.08874,1.08806,1.08924,1.08786,1.08867,1.08820,1.08940,1.08801,1.08882
